In [1]:
import pandas as pd

import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, classification_report

C:\Users\Kuba\Documents\py_interpreter2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = pd.read_csv("data/subtask-2-german/train_de.tsv",sep='\t')
dev_data = pd.read_csv("data/subtask-2-german/dev_de.tsv", sep='\t')


In [3]:
# Mapping label strings to integers
label_map = {"OBJ": 0, "SUBJ": 1}
train_data['label'] = train_data['label'].map(label_map)
print(train_data['label'])

dev_data['label'] = dev_data['label'].map(label_map)
print(dev_data['label'])

0      0
1      0
2      0
3      0
4      0
      ..
795    0
796    0
797    0
798    0
799    0
Name: label, Length: 800, dtype: int64
0      0
1      1
2      0
3      0
4      0
      ..
195    0
196    1
197    1
198    1
199    0
Name: label, Length: 200, dtype: int64


In [4]:
#train_data = train_data.drop('solved_conflict', axis=1)

In [5]:
train_data

sentence_id  \
0    4f9c8bcd60318b0d1257f35ebc7c4ede9f7930e1   
1    0531b165e42997e8eecbb84d1e774c728041db8c   
2    c34bac7a38d7959b0d3c340810f5d2ac3187a3ec   
3    3c6b7daf4e0cce25f45c8497da669c8936a66113   
4    9d241c839c8b801bdbc526cd6e20f70ecd11f341   
..                                        ...   
795  26742376756a241814a3e089349d1c08dd267012   
796  6c68576e3227cd0ae611abe6527e441c196a4e6d   
797  5c2da5d7dc995638e388fa7c0a8b643bdd266a18   
798  7f2b3c069add8f1db904077337748b6c79d659be   
799  c689bd94d876c250724f1e1c6bec84aabff923ea   

                                              sentence  label  
0    Die Ausbreitung des Virus sei beschränkter als...      0  
1      Zwar sei ein Anstieg der Zahlen zu verzeichnen.      0  
2    Der Arbeitsmarkt hat sich so zugunsten der Lei...      0  
3    Es wird das PCR-Pooling mit Antigen-Schnelltes...      0  
4    Als er bemerkte, dass Internetbetreiber die Ge...      0  
..                                                 ...    ...  
795  Der lettische Regierungschef Krisjans Karins h...      0  
796  Das Angiotensin-konvertierende Enzym 2 wird ha...      0  
797  „In enger Abstimmung mit dem Ministerium für A...      0  
798  Unter anderem sollen die Patrouillen an Land u...      0  
799  Am Mittwoch der vergangenen Woche waren zunäch...      0  

[800 rows x 3 columns]

In [6]:
# Load tokenizer
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')

In [7]:
train_encodings = tokenizer(train_data['sentence'].tolist(), truncation=True, padding=True)
eval_encodings = tokenizer(dev_data['sentence'].tolist(), truncation=True, padding=True)

In [8]:
#train_encodings["input_ids"]
#train_encodings["attention_mask"]


In [9]:
# Create PyTorch datasets
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_data['label'])
)

val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(eval_encodings['input_ids']),
    torch.tensor(eval_encodings['attention_mask']),
    torch.tensor(dev_data['label'])
)

In [10]:
print(train_dataset[0][0].shape) # input_ids shape
print(train_dataset[0][1].shape) # attention_mask shape
print(train_dataset[0][2].shape) # train_labels_onehot shape

torch.Size([157])
torch.Size([157])
torch.Size([])


In [11]:
# Define model
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-large', num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# trainer parameters
epochs = 3
learning_rate=5e-5
weight_decay = 0.01 # default 0.01

In [13]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=10,
    learning_rate=learning_rate,
)

In [14]:
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=lambda data: {'input_ids': torch.stack([item[0] for item in data]),
                                'attention_mask': torch.stack([item[1] for item in data]),
                                'labels': torch.stack([item[2] for item in data])},
)

C:\Users\Kuba\Documents\py_interpreter2\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
# Train the model
trainer.train()

# epoch 4 / 50 = 200	0.538800	0.644553

TrainOutput(global_step=150, training_loss=0.6459277629852295, metrics={'train_runtime': 1738.7564, 'train_samples_per_second': 1.38, 'train_steps_per_second': 0.086, 'total_flos': 685843237699200.0, 'train_loss': 0.6459277629852295, 'epoch': 3.0})

In [16]:
#model.save_pretrained("./model")

In [17]:
#model = XLMRobertaForSequenceClassification.from_pretrained("./model")

In [18]:
# test
test_data = pd.read_csv("data/subtask-2-german/dev_test_de.tsv", sep='\t')  # Update with your dev data file
test_data['label'] = test_data['label'].map(label_map)
test_encodings = tokenizer(test_data['sentence'].tolist(), truncation=True, padding=True)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_data['label'])
)

In [19]:
preds = trainer.predict(test_dataset)
pred_labels = preds.predictions.argmax(-1)



In [20]:
print(pred_labels)

[1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0
 1 1 0 0 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0
 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0
 1 0 1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0
 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0
 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0]


In [21]:
# Convert the true labels to integers

# Compute the accuracy and classification report
accuracy = accuracy_score(test_data['label'], pred_labels)
class_report = classification_report(test_data['label'], pred_labels, target_names=['OBJ', 'SUBJ'])

print(f"Accuracy for RoBERTa: {accuracy}")
print(f"Classification Report:\n{class_report}")

Accuracy for RoBERTa: 0.8075601374570447
Classification Report:
              precision    recall  f1-score   support

         OBJ       0.83      0.89      0.86       194
        SUBJ       0.75      0.64      0.69        97

    accuracy                           0.81       291
   macro avg       0.79      0.77      0.77       291
weighted avg       0.80      0.81      0.80       291



In [22]:
# epochs 3 - acc 0.81 - OBJ 0.83, SUBJ 0.75 - f1 0.86 - 0.69
#Step	Training Loss	Validation Loss
#50	0.666900	0.651997
#100	0.747900	0.688412
#150	0.563600	0.500711